In [59]:
from lark import Lark, Token
from lark.tree import Tree
from typing import Callable, List, TypeAlias
from rich import print as rich_print
from contextlib import redirect_stdout

State: TypeAlias = Callable

grammar = None
with open('small.lark') as f:
    grammar = f.read()

parser = Lark(grammar, start='program', parser='lalr')

program = None
with open ('program.small') as f:
    program = f.read()

tree = parser.parse(program)

with open('program.small.tree', 'w') as out:
        print(tree.pretty(), file=out)
        #print(rich_print(tree))
    #out.write(tree.pretty())


In [60]:
def printToken(token: Token, state: State):
    match token.type:
        case 'IDENTIFIER':
            print(state(token))
        case 'NUMBER':
            print(token.value)
        case 'STRING':
            print(eval(token.value))

In [64]:
def printArgs(args: List[Token|Tree], state: State):
    for arg in args:
        printToken(arg, state)

In [65]:
def evalExpression(expression: Token | Tree, state: State):
        typeof_exp: Token | Tree = type(expression)
        
        if typeof_exp == Token:
            match expression.type:
                 case 'NUMBER':
                    return int(expression.value)
                 case 'IDENTIFIER':
                    return state(expression)
            
        elif typeof_exp == Tree:
             e1, e2 = expression.children
             e1, e2 = evalExpression(e1, state), evalExpression(e2, state)
             match expression.data:
                 case 'add':  return e1 + e2
                 case 'sub':  return e1 - e2
                 case 'mul':  return e1 * e2
                 case 'div':  return e1 / e2
                 case 'mod':  return e1 % e2
                 case 'idiv': return e1 // e2

def evalLogic(expression: Token | Tree, state: State):
    if expression.data == 'true':  return True
    elif expression.data == 'false': return False
    elif expression.data == 'logical_not': return not evalLogic(expression.children[0], state)
    else:
        e1, e2 = expression.children
        e1, e2 = evalExpression(e1, state), evalExpression(e2, state)
        match expression.data:
            case 'logical_and': return e1 and e2
            case 'logical_or': return e1 or e2
            case 'logical_lt': return e1 < e2
            case 'logical_gt': return e1 > e2
            case 'logical_eq': return e1 == e2
            case 'logical_neq': return e1 != e2
            case 'logical_leq': return e1 <= e2
            case 'logical_geq': return e1 >= e2
                 
def evalProgram(tree, state: State =  lambda _: 0):
    match tree.data:
        case 'break':
            return state
        case 'skip': # skip
            return state
        
        case 'assignment': # x := E
            x, E = tree.children
            value = evalExpression(E, state)
            return (lambda IDENTIFIER: value if (IDENTIFIER == x) else state(IDENTIFIER))
        
        case 'print': #
            for expression in tree.children:
                if type(expression) == Token:
                    printToken(expression, state)
                elif "args" in expression.data:
                    printArgs(expression.children, state) # FIXME
                elif "logical" in expression.data: print(evalLogic(expression, state))
                else: print(evalExpression(expression, state))
            return state
        
        case 'concat': # S1; S2
            S1, S2 = tree.children
            return evalProgram(S2, evalProgram(S1, state))
        
        case 'while':
            guard, SW = tree.children
            new_state = state
            while evalLogic(guard, new_state):
                new_state = evalProgram(SW, new_state)
                # This is kinda sketchy
                if "break" in str(SW): break
                
            return new_state
        
        case 'conditional':
            guard, s1, s2 = tree.children
            logic = evalLogic(guard, state)
            if logic: return evalProgram(s1, state)
            else: return evalProgram(s2, state)
        
        
         
evalProgram(tree);

Hello, World
!
20
21
27
2
4
6
8
10


In [56]:
eval('print("abc")')

abc
